In [42]:
from collections import Counter
import pandas as pd
import numpy as np
import xgboost as xgb
pd.set_option('display.max_colwidth', -1)

In [43]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from utils import UtilsKy, Statistic

In [77]:
# kyw3
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [117]:
# ky9
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9, dtype=str)
db_test = pd.read_csv(UtilsKy.DB_TEST_KY9, dtype=str)
white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

# for prod
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9_FOR_PROD, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KY9_FOR_PROD , dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

# all false
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9_ALL, dtype=str)

# 300th in teach (10%) 
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9_ALL , dtype=str)

In [118]:
db_teach.shape[0]

746536

In [122]:
db_teach[db_teach.status.isin([1, '1', 'true'])].shape[0]

44294

In [124]:
Statistic.get_cb_rate_amount(db_test)

1.45

In [125]:
Statistic.get_cb_rate(db_test)

0.94

In [93]:
COL_FACTORS = [ 'amount', 'bank_currency', 'hour', 'day_of_week', 
               'bin', 'longitude', 'latitude', 'phone_2_norm']

In [94]:
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [95]:
train = db_teach[COL_FACTORS]
test = db_test[COL_FACTORS]

In [96]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [97]:
replace_val = -9999
train.fillna(replace_val)
test.fillna(replace_val)
replace_val

-9999

In [98]:
train = train.values
test = test.values
label = db_teach.status

In [99]:
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [100]:
result_df_amount = None
weight = analyzer_prediction.get_xgb_weight()

In [101]:
weight

array([16, 16, 16, ...,  1,  1,  1])

In [110]:
# 1
# max_depths =[5]
# nrounds = [90]
# etas = [0.2]

# 2
# max_depths =[3]
# nrounds = [120]
# etas = [0.1]

# 3
# max_depths =[5]
# nrounds = [80]
# etas = [0.3]

# 4
# max_depths =[3]
# nrounds = [80]
# etas = [0.2, 0.3, 0.35, 0.4]

# 5
# max_depths =[5]
# nrounds = [100]
# etas = [0.2, 0.3, 0.4]

# 6 
# max_depths =[5]
# nrounds = [100]
# etas = [0.2, 0.3, 0.35, 0.4]

# 7 best 
# max_depths =[5]
# nrounds = [90]
# etas = [0.35]

# 8 currently 
# max_depths =[3]
# nrounds = [120]
# etas = [0.1]

# 9 secondary
max_depths =[5]
nrounds = [80]
etas = [0.3]

In [111]:
for nround in nrounds:
    for eta in etas:
        for max_depth in max_depths:
            config = {'max_depth': max_depth, 'learning_rate': eta
                      , 'n_estimators': nround
                     }
            model = xgb.XGBClassifier(**config)
            
            model.fit(train, label, eval_metric = 'auc', sample_weight=weight)
             
            test_pred = model.predict_proba(test)
            db_test["probability"] = test_pred[:, 1]
            #description = str(config) + 'wl'
            description = '-' . join([str(elem) for elem in (max_depth, nround, eta)])                    
            result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")

In [112]:
result_df_amount#.iloc[:,9:]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,5-90-0.35,4.270000,7.510000,9.370000,11.380000,13.320000,15.150000,16.730000,21.990000,42.290000,...,289780,294698,2.025401e+07,2.083044e+07,746536,44294,603378,5642,660225.66,4.549701e+07
1,threshold amount,0.788145,0.749648,0.722876,0.704270,0.688690,0.675084,0.662167,0.630373,0.543394,...,289780,294698,2.025401e+07,2.083044e+07,746536,44294,603378,5642,660225.66,4.549701e+07
2,3-120-0.1,1.690000,4.690000,6.790000,9.300000,12.180000,14.160000,16.900000,25.870000,48.550000,...,289780,294698,2.025401e+07,2.083044e+07,746536,44294,603378,5642,660225.66,4.549701e+07
3,threshold amount,0.717879,0.696819,0.682698,0.671526,0.663888,0.655805,0.649229,0.631626,0.578514,...,289780,294698,2.025401e+07,2.083044e+07,746536,44294,603378,5642,660225.66,4.549701e+07
4,5-80-0.3,4.190000,6.840000,9.040000,11.080000,12.760000,14.910000,17.200000,24.220000,45.330000,...,289780,294698,2.025401e+07,2.083044e+07,746536,44294,603378,5642,660225.66,4.549701e+07
5,threshold amount,0.766547,0.732861,0.712230,0.696252,0.682646,0.670515,0.659859,0.632589,0.553287,...,289780,294698,2.025401e+07,2.083044e+07,746536,44294,603378,5642,660225.66,4.549701e+07
6,5-80-0.3,4.190000,6.840000,9.040000,11.080000,12.760000,14.910000,17.200000,24.220000,45.330000,...,289780,294698,2.025401e+07,2.083044e+07,746536,44294,603378,5642,660225.66,4.549701e+07
7,threshold amount,0.766547,0.732861,0.712230,0.696252,0.682646,0.670515,0.659859,0.632589,0.553287,...,289780,294698,2.025401e+07,2.083044e+07,746536,44294,603378,5642,660225.66,4.549701e+07


In [22]:
#result_df_amount

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,5-0.35-90,2.280000,6.270000,7.390000,7.990000,9.790000,12.430000,16.740000,24.860000,39.16000,...,92012,93565,6.248312e+06,6.473538e+06,4006662,54811,299113,1679,240114.61,25077725.29
1,threshold amount,0.824859,0.796214,0.777232,0.762211,0.748784,0.737005,0.727927,0.701785,0.63337,...,92012,93565,6.248312e+06,6.473538e+06,4006662,54811,299113,1679,240114.61,25077725.29


In [73]:
# result_df_amount

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,5-0.35-90,5.140000,8.810000,11.510000,14.33000,15.990000,18.100000,19.540000,25.460000,46.310000,...,289780,294698,2.025401e+07,2.083044e+07,742167,42005,603378,5642,660225.66,4.549701e+07
1,threshold amount,0.781048,0.742247,0.719417,0.70153,0.687229,0.674351,0.662386,0.631375,0.545214,...,289780,294698,2.025401e+07,2.083044e+07,742167,42005,603378,5642,660225.66,4.549701e+07


In [113]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ]
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)

stat_best = stat_best.sort_values(by="rating", ascending=False)
stat_best = stat_best.sort_values(by="p_5", ascending=False)

In [116]:
stat_best.iloc[:,0:11]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
0,5-90-0.35,4.27,7.51,9.37,11.38,13.32,15.15,16.73,21.99,42.29,45.85
4,5-80-0.3,4.19,6.84,9.04,11.08,12.76,14.91,17.20,24.22,45.33,43.91
6,5-80-0.3,4.19,6.84,9.04,11.08,12.76,14.91,17.20,24.22,45.33,43.91
2,3-120-0.1,1.69,4.69,6.79,9.30,12.18,14.16,16.90,25.87,48.55,34.65


,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,5-0.2-90,4.450000,6.520000,8.490000,10.610000,13.140000,15.69000,17.930000,26.600000,47.850000,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
1,threshold amount,0.760708,0.723037,0.703265,0.687484,0.674667,0.66293,0.653549,0.629152,0.558903,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07


In [55]:
# stat_best.iloc[:,0:11]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
18,5-0.35-90,5.14,8.81,11.51,14.33,15.99,18.10,19.54,25.46,46.31,55.78
26,5-0.35-100,5.11,8.69,11.08,14.25,15.72,17.50,19.46,25.00,45.83,54.85
28,5-0.4-100,3.71,6.31,9.49,12.81,15.33,17.33,19.64,26.72,47.20,47.65
8,5-0.3-80,4.20,6.97,9.39,11.68,14.91,16.65,18.00,24.93,47.57,47.15
16,5-0.3-90,4.00,6.22,8.83,11.85,14.76,17.41,18.75,25.31,47.92,45.66
12,3-0.35-80,3.78,6.80,8.99,11.80,14.63,17.15,19.31,25.61,46.72,46.00
24,5-0.3-100,3.52,6.16,9.07,11.61,14.56,16.54,19.25,26.16,46.67,44.92
20,5-0.4-90,3.51,6.00,9.11,11.96,14.54,16.92,19.88,27.02,47.50,45.12
6,5-0.25-80,4.67,7.53,9.61,11.71,14.33,15.78,17.89,23.78,48.65,47.85
4,5-0.2-80,4.53,6.49,8.67,11.08,13.86,16.73,18.64,26.60,47.41,44.63
